In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline,RobertaForMaskedLM,RobertaTokenizer
from transformers import BitsAndBytesConfig
from peft import PeftModelForCausalLM
import torch

In [2]:
from vllm import LLM, SamplingParams,EngineArgs, LLMEngine, RequestOutput
from vllm.lora.request import LoRARequest


In [3]:
engine_args = EngineArgs(model="deepseek-ai/deepseek-coder-6.7b-base",
                             enable_lora=True,
                             max_loras=1,
                             max_lora_rank=8,
                             max_cpu_loras=2,
                             max_num_seqs=256,
                             max_model_len= 512)
engine = LLMEngine.from_engine_args(engine_args)

INFO 02-28 16:24:36 llm_engine.py:72] Initializing an LLM engine with config: model='deepseek-ai/deepseek-coder-6.7b-base', tokenizer='deepseek-ai/deepseek-coder-6.7b-base', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=512, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 02-28 16:24:39 weight_utils.py:164] Using model weights format ['*.bin']
INFO 02-28 16:24:47 llm_engine.py:322] # GPU blocks: 503, # CPU blocks: 512
INFO 02-28 16:24:48 model_runner.py:632] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-28 16:24:48 model_runner.py:636] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 02-28 16:24:53 model_runner.py:698] Graph capturing finished in 5 secs.


In [4]:
lorarequest = LoRARequest("DeepSeek7bForCodeTrans",1,"DeepSeek7bForCodeTrans/checkpoint-6435")

In [5]:
engine.add_lora(lorarequest)

[True]

In [6]:
additional_special_tokens = {'additional_special_tokens':['<|begin_of_java_code|>','<|end_of_java_code|>'\
                                                           ,'<|begin_of_c-sharp_code|>','<|end_of_c-sharp_code|>',\
                                                            '<|translate|>']}
prompt = "public void serialize(LittleEndianOutput out) {out.writeShort(field_1_vcenter);}\n"
prompt = additional_special_tokens['additional_special_tokens'][0] + prompt + additional_special_tokens['additional_special_tokens'][1] + additional_special_tokens['additional_special_tokens'][2]
prompt

'<|begin_of_java_code|>public void serialize(LittleEndianOutput out) {out.writeShort(field_1_vcenter);}\n<|end_of_java_code|><|begin_of_c-sharp_code|>'

In [7]:
sampling_params = SamplingParams(temperature=0.1,max_tokens= 512,stop_token_ids=[32022,32014],skip_special_tokens=False)
engine.add_request(str(3),prompt,sampling_params,lora_request=lorarequest)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
engine.step()

INFO 02-28 16:24:57 llm_engine.py:877] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 1 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.4%, CPU KV cache usage: 0.0%


[RequestOutput(request_id=3, prompt='<|begin_of_java_code|>public void serialize(LittleEndianOutput out) {out.writeShort(field_1_vcenter);}\n<|end_of_java_code|><|begin_of_c-sharp_code|>', prompt_token_ids=[32013, 32022, 3566, 2494, 11071, 938, 7, 29329, 6015, 987, 8780, 631, 8, 507, 406, 13, 6449, 19721, 7, 3267, 62, 16, 62, 85, 4679, 477, 92, 185, 32023, 32024], prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='public', token_ids=[3566], cumulative_logprob=0.0, logprobs=None, finish_reason=None)], finished=False, lora_request=LoRARequest(lora_name='DeepSeek7bForCodeTrans', lora_int_id=1, lora_local_path='DeepSeek7bForCodeTrans/checkpoint-6435'))]

In [25]:
real_output = ""
finished = False
while engine.has_unfinished_requests():
    request_outputs  =  engine.step()
    for request_output in request_outputs:
        finished = finished | request_output.finished

INFO 02-28 16:31:15 llm_engine.py:877] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 124.4 tokens/s, Running: 3 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 5.4%, CPU KV cache usage: 0.0%


In [28]:
request_outputs[0].outputs[0].token_ids

[3566,
 6632,
 5891,
 14361,
 5349,
 50,
 5492,
 19467,
 31324,
 6522,
 5891,
 14361,
 5349,
 50,
 5492,
 19467,
 31324,
 7,
 2245,
 14361,
 5349,
 50,
 5492,
 19467,
 31324,
 4397,
 3092,
 4546,
 1881,
 3438,
 405,
 756,
 21968,
 3507,
 7878,
 1293,
 6732,
 13,
 4397,
 9468,
 11862,
 250,
 405,
 5891,
 14361,
 5349,
 50,
 5492,
 19467,
 31324,
 4397,
 9468,
 11862,
 250,
 13,
 7552,
 26,
 6732,
 13,
 6522,
 2826,
 3957,
 11862,
 250,
 405,
 5891,
 14361,
 5349,
 50,
 5492,
 19467,
 31324,
 6522,
 2826,
 3957,
 11862,
 250,
 13,
 7552,
 26,
 2125,
 21968,
 3507,
 27,
 2245,
 14361,
 5349,
 50,
 5492,
 19467,
 31324,
 6522,
 29,
 7,
 6555,
 11,
 3438,
 477,
 92,
 185,
 32022]

In [26]:
request_outputs[0].outputs[0].text

'public virtual ListSpeechSynthesisTasksResponse ListSpeechSynthesisTasks(ListSpeechSynthesisTasksRequest request){var options = new InvokeOptions();options.RequestMarshaller = ListSpeechSynthesisTasksRequestMarshaller.Instance;options.ResponseUnmarshaller = ListSpeechSynthesisTasksResponseUnmarshaller.Instance;return Invoke<ListSpeechSynthesisTasksResponse>(request, options);}\n<|begin_of_java_code|>'

In [20]:
additional_special_tokens = {'additional_special_tokens':['<|begin_of_java_code|>','<|end_of_java_code|>'\
                                                           ,'<|begin_of_c-sharp_code|>','<|end_of_c-sharp_code|>',\
                                                            '<|translate|>']}
prompt = "public ListSpeechSynthesisTasksResult listSpeechSynthesisTasks(ListSpeechSynthesisTasksRequest request) {request = beforeClientExecution(request);return executeListSpeechSynthesisTasks(request);}\n"
prompt = additional_special_tokens['additional_special_tokens'][-1] + prompt + additional_special_tokens['additional_special_tokens'][3] + additional_special_tokens['additional_special_tokens'][2]
prompt

'<|translate|>public ListSpeechSynthesisTasksResult listSpeechSynthesisTasks(ListSpeechSynthesisTasksRequest request) {request = beforeClientExecution(request);return executeListSpeechSynthesisTasks(request);}\n<|end_of_c-sharp_code|><|begin_of_c-sharp_code|>'

In [23]:
sampling_params = SamplingParams(temperature=0.1,max_tokens= 512,stop_token_ids=[32025,32014],skip_special_tokens=False)
engine.add_request(str(5),prompt,sampling_params,lora_request=lorarequest)

In [24]:
engine.step()

INFO 02-28 16:31:10 llm_engine.py:877] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 3 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 2.4%, CPU KV cache usage: 0.0%


[RequestOutput(request_id=5, prompt='<|translate|>public ListSpeechSynthesisTasksResult listSpeechSynthesisTasks(ListSpeechSynthesisTasksRequest request) {request = beforeClientExecution(request);return executeListSpeechSynthesisTasks(request);}\n<|end_of_c-sharp_code|><|begin_of_c-sharp_code|>', prompt_token_ids=[32013, 32026, 3566, 5891, 14361, 5349, 50, 5492, 19467, 31324, 4835, 1517, 14361, 5349, 50, 5492, 19467, 31324, 7, 2245, 14361, 5349, 50, 5492, 19467, 31324, 4397, 3092, 8, 507, 6555, 405, 1321, 5960, 25793, 7, 6555, 477, 2125, 11682, 2245, 14361, 5349, 50, 5492, 19467, 31324, 7, 6555, 477, 92, 185, 32025, 32024], prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='public', token_ids=[3566], cumulative_logprob=0.0, logprobs=None, finish_reason=None)], finished=False, lora_request=LoRARequest(lora_name='DeepSeek7bForCodeTrans', lora_int_id=1, lora_local_path='DeepSeek7bForCodeTrans/checkpoint-6435'))]

In [10]:
def match_c_sharp_code(code:str):
    codelst = code.split('<|begin_of_java_code|>')
    return codelst[0]

In [11]:
code = "public override void Serialize(ILittleEndianOutput out1){out1.WriteShort(field_1_vcenter);}<|begin_of_java_code|>"

In [12]:
match_c_sharp_code(code)

'public override void Serialize(ILittleEndianOutput out1){out1.WriteShort(field_1_vcenter);}'